In [2]:
import numpy as np
import pandas as pd
%matplotlib inline
# for plotting
import seaborn as sns
sns.set(style='white', font_scale=1.5)
#for working with Bam files
import pysam
import matplotlib.pyplot as plt
import matplotlib.backends.backend_pdf
#for distance
from scipy.spatial import distance
import datetime
from mpl_toolkits.mplot3d import Axes3D

import logging
from gridmap import grid_map


In [3]:
cd /broad/mushroom_lasagna/YB/Ht115_lineage_on_itself/

/broad/mushroom_lasagna/YB/Ht115_lineage_on_itself


In [4]:
################################################################
#create the header file _ by using the original output file
frames=[]
out_files = !ls | grep 'T'
f=out_files[3]
print f
!grep contig {f} > /broad/mushroom_lasagna/YB/{'header2.txt'} 
################################################################

T34_N48.txt


In [5]:
!head -n 5 T34_N48.txt

1	76419	TAAxGCC	T	G	T34	N48	0	NOVEL	COVERED	0.954491	0.954491	1	1	0.999912	141	55	0.303488	7.938785	-0.240667	9.165872	0.104167	0.02	-0.420221	47	43	5	1236	124	40	37	0	0	TT	15.847652	0.015625	32	63	1	1743	29	0.794929	0.88241	(27,16,1,4)	20	10	129	11	0		KEEP
1	99014	ATTxATT	T	C	T34	N48	0	NOVEL	COVERED	1	1	1	1	1	249	2	-17.123764	9.758706	4.941053	4.84515	0.048387	0.02	2.682436	131	118	6	3311	173	60	45	2	0	TT	29.601009	0.009091	107	109	1	3055	30	0.876241	0.784668	(50,68,3,3)	117.5	4.5	28	0.5	0		KEEP
1	1131218	TCTxCCT	C	T	T34	N48	0	NOVEL	COVERED	0.941741	0.941741	1	1	0.999974	85	0	0.283412	7.952986	1.656884	6.468233	0.088889	0.02	-0.411888	46	41	4	1188	119	60	60	0	0	CC	11.367335	0	36	38	0	1058	0	0.774405	0.798238	(22,19,1,3)	58.5	12	91.5	12	0		KEEP
1	1137744	CCCxAGC	G	C	T34	N48	0	NOVEL	UNCOVERED	0.138793	0.139832	0.99257	0.99257	0	35	15	6.842018	6.871529	1.080468	6.134259	0.444444	0.02	-0.094677	9	5	4	125	77	31	32	0	0	GG	4.465581	0.052632	6	18	1	419	8	0.983402	0.62965	(0,5,1,3)	112	24.5	35

In [5]:
#load all files into one big table - sort by 
frames=[]
out_files = !ls | grep 'T'
count=1
columns = pd.read_csv('/broad/mushroom_lasagna/YB/header.txt', sep='\t', comment='#').columns
for file in out_files:
    wt = pd.read_csv(file, sep='\t')
    wt = pd.DataFrame(wt.as_matrix(), columns=columns)
    print ("for: ",file," -The size is: ",wt.shape)
    if (count==1):
        wf=wt
    else:
        wf=wf.append(wt, ignore_index=True)
    count=count+1
print (wf.shape)

('for: ', 'T34_N37.txt', ' -The size is: ', (13222, 51))
('for: ', 'T34_N45.txt', ' -The size is: ', (5023, 51))
('for: ', 'T34_N47.txt', ' -The size is: ', (5093, 51))
('for: ', 'T34_N48.txt', ' -The size is: ', (4812, 51))
('for: ', 'T34_N49.txt', ' -The size is: ', (5029, 51))
('for: ', 'T34_N54.txt', ' -The size is: ', (5193, 51))
('for: ', 'T34_N63.txt', ' -The size is: ', (5130, 51))
('for: ', 'T37_N34.txt', ' -The size is: ', (36750, 51))
('for: ', 'T37_N45.txt', ' -The size is: ', (36775, 51))
('for: ', 'T37_N47.txt', ' -The size is: ', (36811, 51))
('for: ', 'T37_N48.txt', ' -The size is: ', (36801, 51))
('for: ', 'T37_N49.txt', ' -The size is: ', (37411, 51))
('for: ', 'T37_N54.txt', ' -The size is: ', (36701, 51))
('for: ', 'T37_N63.txt', ' -The size is: ', (37363, 51))
('for: ', 'T45_N34.txt', ' -The size is: ', (3078, 51))
('for: ', 'T45_N37.txt', ' -The size is: ', (11439, 51))
('for: ', 'T45_N47.txt', ' -The size is: ', (3414, 51))
('for: ', 'T45_N48.txt', ' -The size is

/broad/software/free/Linux/redhat_6_x86_64/pkgs/anaconda_2.1.0/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2871: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
cd /seq/picard_aggregation/G101103/

In [ ]:
cwf=wf.drop_duplicates(['position','contig'])

In [ ]:
f1=open('/broad/mushroom_lasagna/YB/Ht115_lineage_on_itself/log.txt', 'w+')
co=1
columns=('contig','position','context','ref_all','alt_all','t_lod_fstar','vector','distance_in_g1','distance_in_g2','per_bet_gr','real_bet_gr','ref_af','alt_af','type','full_data')
mutation=[]
f1.write("Log for run: /n")
keep=['contig', 'position','context', 'ref_allele','alt_allele','t_lod_fstar']
locations=cwf[keep]

In [ ]:
print locations.shape

In [ ]:
#This section get table with locations and for each location add the spot in all the samples
#and save the table - should run only once.
#def mat_from_bam(locations):
# this have to be in seperate line - ??????
#!cd /seq/picard_aggregation/G101103/
f1=open('/broad/mushroom_lasagna/YB/Ht115_lineage_on_itself/log.txt', 'w+')
f1.write("Log for run: /n")

head=[]
begin=!pwd 
begin=begin.pop()
out_files = !ls | grep HT6
    #prepare list of all bam files to read
    #open bam file and load all the locations - and add to the table
for f in out_files:
    path="%s/%s/v4/%s.bam"%(begin,f,f)
    head.append(f)
    samfile = pysam.AlignmentFile(path, "rb")
        #Samples.append(samfile)
        #for every bam file run over all the mutations
    seq=[]
    f1.write("Reading from file: %s\n"%(f))
    f1.flush()
    for i in locations.index:
        if (i%500 ==0):
            f1.write("file %s finished = %i lines\n" %(f,i))
            f1.flush()
        s=""
        chr= str(locations.contig[i])
        region= int(float(locations.position[i]))
        region=region-1
        for row in samfile.pileup(chr,region,region+1):
                if row.pos ==region:
                    for row in row.pileups:
                        if (row.query_position is not None):
                            #print (row.query_position)
                            #print (row.alignment.query_sequence[row.query_position])
                            s+=row.alignment.query_sequence[row.query_position]
        s=''.join(sorted(s.upper()))
        seq.append(s)
        # add the seq as column in the db - location
    #locations[f]=pd.Series(seq,index=f)
    #print "f= ",f," len seq= ",seq
    locations[f]=seq

########################
#and save the new table
locations.to_pickle('/broad/mushroom_lasagna/YB/Ht115_lineage_on_itself/mutations.pkl')
f1.write("file %s finished = %i lines\n" %(f,i))
f1.flush()
close(f1)
f1.write("finito")